In [1]:
import os
import glob

import DESutils
import GPRutils
import vK2KGPR
import plotGPR
import vK2KGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import astropy.table as tb
from astropy.time import Time
import matplotlib.pyplot as plt
plt.style.use('~/GitHub/custom-matplotlib/custom.mplstyle')

from IPython import embed
from importlib import reload
import warnings
warnings.filterwarnings("ignore")

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [3]:
for expNum in DESutils.createBandDict("/data3/garyb/tno/y6/zone133")[0]:
    files = {
#         "RCmax": f"/home/fortino/GPRSolutions/dualOpt_NMmax/GPR.{expNum}.?RC",
#         "RCLmax": f"/home/fortino/GPRSolutions/L_BFGS_B_max/GPR.{expNum}.?RC",
#         "RCLmax2": f"/home/fortino/GPRSolutions/L_BFGS_B_max2/GPR.{expNum}.?RC",
#         "Eris_RCLmax": f"/home/fortino/GPRSolutions/eris/GPR.{expNum}.?RC",
        "zone": f"/home/fortino/GPRSolutions/zone133/GPR.{expNum}.?RC",
    }
    
    for i, (name, file) in enumerate(files.items()):
        if len(glob.glob(file+".fits")) == 0:
            complete = False
            break
    else:
        complete = True
    
    if not complete:
        continue
    
    dC0 = GPRutils.loadFITS(list(files.values())[0] + ".fits")
    print(f"{expNum}, {dC0.band}")
    names = ["Method", "xi0", "xif", "Reduction", "nOpt1", "nOpt2", "totalTime", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    names = ["Method", "xi0", "xif", "Reduction", "Stars/arcmin", "totalTime", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    header = "".join([f"{name:<15}" for name in names])
    print(header)
    
    for i, (name, file) in enumerate(files.items()):
        dC = GPRutils.loadFITS(file + ".fits")

        vals = [name]
        
        xi0 = dC.header["xi0"]
        Xerr = dC.header["xi0_Xerr"]
        Yerr = dC.header["xi0_Yerr"]
        xi0err = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xi0:.2f} ± {xi0err:.2f}")

        xif = dC.header["xif"]
        Xerr = dC.header["xif_Xerr"]
        Yerr = dC.header["xif_Yerr"]
        xiferr = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xif:.2f} ± {xiferr:.2f}")

        red = xi0/xif
        rederr = np.sqrt(((xi0err/xi0)**2 + (xiferr/xif)**2) * red**2)
        vals.append(f"{red:.2f} ± {rederr:.2f}")
        
        N = len(dC.TV[dC.TV["Maskf"]])
        starDensity = np.round((N / (3*u.deg**2)).to(u.arcmin**-2), 3)
        vals.append(starDensity.value)
        
        OUTfile = glob.glob(file + ".out")[0]
        parseOut = DESutils.parseOutfile(OUTfile)
        if parseOut.finished:
#             vals.append(parseOut.nOpt1)
#             vals.append(parseOut.nOpt2)
            vals.append(parseOut.totalTime.value)
        else:
#             vals.extend([-1, -1, -1])
            vals.extend([-1])
        
        vals.extend(np.abs(np.around(dC.params, 7)))

        line = "".join([f"{param:<15}" for param in vals])
        print(line)
    print()
    
# a = b/c
# (da/a)^2 = (db/b)^2 + (dc/c)^2

254134, Y
Method         xi0            xif            Reduction      Stars/arcmin   totalTime      K Variance     Outer Scale    Diameter       Wind X         Wind Y         
zone           336.43 ± 5.41  15.55 ± 1.45   21.63 ± 2.05   1.096          4.212          565.379162     3.2765911      0.0184859      0.0175921      0.0318116      

352883, g
Method         xi0            xif            Reduction      Stars/arcmin   totalTime      K Variance     Outer Scale    Diameter       Wind X         Wind Y         
zone           62.82 ± 5.09   23.93 ± 4.80   2.63 ± 0.57    0.878          4.08           61.7341373     0.5185054      0.01           0.0824658      0.030097       

354913, z
Method         xi0            xif            Reduction      Stars/arcmin   totalTime      K Variance     Outer Scale    Diameter       Wind X         Wind Y         
zone           136.71 ± 2.28  7.36 ± 0.87    18.58 ± 2.23   1.242          17.455         289.2668587    2.0738919      0.01           0.0